In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [2]:
train=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv",parse_dates=['date'])
test=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv",parse_dates=['date'])
oil=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/oil.csv")
sample_submission=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv")
holiday_events=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv")
stores=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/stores.csv")
transactions=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/transactions.csv")

In [3]:

# test.head()

In [4]:

train['year']=train['date'].dt.year
train['month']=train['date'].dt.month
train['day']=train['date'].dt.day
train['weekday']=train['date'].dt.weekday
# train=train.drop(columns=['date'])

y=train.sales
X=train.select_dtypes(include=['number']).drop(columns=['sales'])


In [5]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42)


In [6]:

model=XGBRegressor(n_jobs=-1,
                   random_state=42,
                  n_estimators=500,
                  max_depth=7,
                  learning_rate=0.1,
                  reg_alpha=1,
                  reg_lambda=1,
                  subsample=0.7,
                  colsample_bytree=0.8)
# params={
#     'n_estimators':[100,500,1000],
#     'max_depth':[3,5,7],
#     'reg_lambda':[0,0.1,1],
#     'reg_alpha':[0,0.1,1],
#     'learning_rate':[0.1,0.05,0.01],
#     'subsample':[0.7,0.8,1],
#     'colsample_bytree':[0.7,0.8,1]
# }
# r_search=RandomizedSearchCV(
#     estimator=model,
#     param_distributions=params,
#     cv=3,
#     scoring='neg_mean_absolute_error',
#     verbose=1,
#     n_jobs=-1
    
# )
# r_search.fit(X_train,y_train)
# best_model=r_search.best_estimator_
# pred=best_model.predict(X_test)
model.fit(X_train,y_train)
pred=model.predict(X_test)
# print("best parameters",r_search.best_params_)
print("abs error",mean_absolute_error(y_test,pred))
print("squared error",mean_squared_error(y_test,pred))
print("r2 score",r2_score(y_test,pred))


abs error 313.4310227153571
squared error 576344.1663782502
r2 score 0.532485780073731


In [7]:
test['year']=test['date'].dt.year
test['month']=test['date'].dt.month
test['day']=test['date'].dt.day
test['weekday']=test['date'].dt.weekday
main_test=test.select_dtypes(include=['number'])

main_pred=model.predict(main_test)


In [8]:
submission=pd.DataFrame({
    'id':test.id,
    'sales':main_pred
})
submission.to_csv("submission.csv",index=False)

In [9]:
# transactions.head()

In [10]:
# oil.head()

In [11]:
# holiday_events.head()

In [12]:
# stores.head()

In [13]:
sample_submission.head()

,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,0.0
4,3000892,0.0
